In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect, func
import time
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

online_errors = 0
retail_errors = 0
key_errors = 0
marketing_errors = 0
# inspector = inspect(engine)
# inspector.get_table_names()

In [2]:
online_db = pd.read_csv("Resources/Online.csv")
retail_db = pd.read_csv("Resources/Retail.csv")
key_db = pd.read_csv("Resources/KEY_SKU.csv",encoding='utf-8-sig')
marketing_db =pd.read_csv("Resources/Marketing_Spend.csv")
online_start_time = time.time()
retail_start_time = time.time()
key_start_time = time.time()
marketing_start_time = time.time()
# engine = create_engine("sqlite:///Resources/Online.sqlite", echo=False)
# engine2 = create_engine("sqlite:///Resources/key.sqlite", echo=False)
# engine3 = create_engine("sqlite:///Resources/retail.sqlite", echo=False)
# engine4 = create_engine("sqlite:///Resources/marketing.sqlite", echo=False)
engine = create_engine("sqlite:///Resources/data.sqlite", echo=False)





In [3]:
online_db.head()

,Transaction ID,Date,Product SKU,Product,Product Category (Enhanced E-commerce),Quantity,Avg. Price,Revenue,Tax,Delivery
0,48497,20171231,GGOENEBQ079099,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4.0,80.52,316.00,34.44,19.99
1,48496,20171231,GGOENEBQ079199,Nest® Protect Smoke + CO White Wired Alarm-USA,Nest-USA,5.0,80.52,395.00,33.14,6.50
2,48495,20171231,GGOENEBQ084699,Nest® Learning Thermostat 3rd Gen-USA - White,Nest-USA,1.0,151.88,149.00,12.06,6.50
3,48494,20171231,GGOEGAEB091117,Google Zip Hoodie Black,Apparel,1.0,48.92,47.99,3.82,6.50
4,48493,20171231,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,Nest-USA,1.0,121.30,119.00,8.31,6.50


In [4]:
retail_db.head()

,InvoiceNo,InvoiceDate,StockCode,Quantity
0,536598,2017-01-01,21421,1
1,536598,2017-01-01,21422,2
2,536598,2017-01-01,22178,26
3,536599,2017-01-01,20749,2
4,536599,2017-01-01,21056,2


In [5]:
key_db.head()

,Product SKU,StockCode
0,GGOENEBQ079099,21421
1,GGOENEBQ079199,21422
2,GGOENEBQ084699,22178
3,GGOEGAEB091117,20749
4,GGOENEBB078899,21056


In [6]:
marketing_db.head()

,Unnamed: 0,Offline Spend,Online Spend
0,2017-01-01,4500,2424.50
1,2017-01-02,4500,3480.36
2,2017-01-03,4500,1576.38
3,2017-01-04,4500,2928.55
4,2017-01-05,4500,4055.30


In [ ]:
# inner join key to retail on stock code
#then inner join new retail_db on to online_db on product sku
#then on marketing join on invoice date after changing the name

In [7]:
marketing_db['Unnamed: 0']=pd.to_datetime(marketing_db['Unnamed: 0'].astype(str), format='%Y/%m/%d')


In [29]:

# try:
#     marketing_trans = marketing_db.rename(columns={'Unnamed:0':'Invoice_Date',
#                                                       'Offline Spend':'Offline Budget',
#                                                       'Online Spend':'Online Budget'})
# except:    
#     marketing_errors+=1
# marketing_trans.head()

,Unnamed: 0,Offline Budget,Online Budget
0,2017-01-01,4500,2424.50
1,2017-01-02,4500,3480.36
2,2017-01-03,4500,1576.38
3,2017-01-04,4500,2928.55
4,2017-01-05,4500,4055.30


In [30]:
try:
    marketing_transf= marketing_db.rename(columns={"Unnamed: 0": "Invoice Date", 
                                                                 "Offline Spend":"Offline Budget", 
                                                                 "Online Spend":"Online Budget"})
    
except:
    marketing_errors+=1
marketing_transf.head()

,Invoice Date,Offline Budget,Online Budget
0,2017-01-01,4500,2424.50
1,2017-01-02,4500,3480.36
2,2017-01-03,4500,1576.38
3,2017-01-04,4500,2928.55
4,2017-01-05,4500,4055.30


In [13]:
marketing_errors

1

In [20]:
try:
    key_trans = key_db.rename(columns={'Product SKU':'Product_Sku',
                                      'StockCode':'Stock Code'})
except:
    key_errors+=1
    
key_trans.head()

,Product_Sku,Stock Code
0,GGOENEBQ079099,21421
1,GGOENEBQ079199,21422
2,GGOENEBQ084699,22178
3,GGOEGAEB091117,20749
4,GGOENEBB078899,21056


In [8]:
#create report count
key_count=key_trans.shape[0]


In [24]:
retail_db['InvoiceDate']=pd.to_datetime(retail_db['InvoiceDate'].astype(str), format='%Y/%m/%d')

try:
    retail_trans = retail_db.rename(columns={"InvoiceNo": "Invoice No", 
                                        "InvoiceDate":"Invoice Date", 
                                        "StockCode":"Stock Code", 
                                        "Quantity":"Quantity"})
except:
    retail_errors+=1
retail_trans.head()

,Invoice No,Invoice Date,Stock Code,Quantity
0,536598,2017-01-01,21421,1
1,536598,2017-01-01,21422,2
2,536598,2017-01-01,22178,26
3,536599,2017-01-01,20749,2
4,536599,2017-01-01,21056,2


In [16]:
retail_count = retail_trans.shape[0]

In [17]:
online_db['Date']=pd.to_datetime(online_db['Date'].astype(str), format='%Y/%m/%d')

try:
    online_trans = online_db.rename(columns={"Transaction ID":"Transaction Id", 
                                             "Date":"Invoice Date", 
                                             "Product SKU":"Product_Sku",
                                             "Product":"Product",
                                             "Product Category (Enhanced E-commerce)":"Product Catagory",
                                             "Quantity":"Quantity",
                                             "Avg. Price":"Average Price",
                                             "Revenue":"Revenue",
                                             "Tax":"Tax",
                                             "Delivery":"Delivery Cost"})
except:
    online_errors+=1
online_trans.head()

,Transaction Id,Invoice Date,Product_Sku,Product,Product Catagory,Quantity,Average Price,Revenue,Tax,Delivery Cost
0,48497,2017-12-31,GGOENEBQ079099,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4.0,80.52,316.00,34.44,19.99
1,48496,2017-12-31,GGOENEBQ079199,Nest® Protect Smoke + CO White Wired Alarm-USA,Nest-USA,5.0,80.52,395.00,33.14,6.50
2,48495,2017-12-31,GGOENEBQ084699,Nest® Learning Thermostat 3rd Gen-USA - White,Nest-USA,1.0,151.88,149.00,12.06,6.50
3,48494,2017-12-31,GGOEGAEB091117,Google Zip Hoodie Black,Apparel,1.0,48.92,47.99,3.82,6.50
4,48493,2017-12-31,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,Nest-USA,1.0,121.30,119.00,8.31,6.50


In [18]:
online_count = online_trans.shape[0]

In [22]:
sku_join = pd.merge(key_trans,retail_trans,on='Stock Code',how = 'inner')
sku_join.set_index('Invoice No', inplace=True)
sku_join.head()

,Product_Sku,Stock Code,Invoice Date,Quantity
Invoice No,,,,
536598,GGOENEBQ079099,21421,2017-01-01,1
536999,GGOENEBQ079099,21421,2017-01-02,1
537137,GGOENEBQ079099,21421,2017-01-04,2
538296,GGOENEBQ079099,21421,2017-01-09,1
540247,GGOENEBQ079099,21421,2017-02-04,1


In [23]:
sku_online_join= pd.merge(key_trans, online_trans, on='Product_Sku',how = 'inner')
sku_online_join.set_index('Product_Sku',inplace=True)
sku_online_join.head()

,Stock Code,Transaction Id,Invoice Date,Product,Product Catagory,Quantity,Average Price,Revenue,Tax,Delivery Cost
Product_Sku,,,,,,,,,,
GGOENEBQ079099,21421,48497,2017-12-31,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4.0,80.52,316.0,34.44,19.99
GGOENEBQ079099,21421,48491,2017-12-31,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,2.0,80.52,376.0,0.00,6.50
GGOENEBQ079099,21421,48475,2017-12-31,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,4.0,80.52,1022.0,92.58,6.50
GGOENEBQ079099,21421,48468,2017-12-31,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,2.0,80.52,158.0,14.59,6.50
GGOENEBQ079099,21421,48466,2017-12-31,Nest® Protect Smoke + CO White Battery Alarm-USA,Nest-USA,3.0,80.52,237.0,21.61,6.50


In [25]:
retail_online_join = pd.merge(retail_trans, online_trans, on='Invoice Date',how='inner')
retail_online_join.head()

,Invoice No,Invoice Date,Stock Code,Quantity_x,Transaction Id,Product_Sku,Product,Product Catagory,Quantity_y,Average Price,Revenue,Tax,Delivery Cost
0,536598,2017-01-01,21421,1,16736,GGOENEBJ079499,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,1.0,153.71,149.0,10.89,6.5
1,536598,2017-01-01,21421,1,16733,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,Nest-USA,1.0,122.77,119.0,11.00,6.5
2,536598,2017-01-01,21421,1,16732,GGOENEBJ079499,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,1.0,153.71,149.0,13.22,6.5
3,536598,2017-01-01,21421,1,16731,GGOENEBQ078999,Nest® Cam Outdoor Security Camera - USA,Nest-USA,2.0,122.77,238.0,20.80,6.5
4,536598,2017-01-01,21421,1,16727,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,Nest-USA,1.0,122.77,119.0,11.00,6.5


In [27]:
cleaned_retail_online = retail_online_join.drop(columns=['Quantity_y'])
cleaned_retail_online = cleaned_retail_online.rename(columns={'Quantity_x':'Quantity'})
cleaned_retail_online.head()

,Invoice No,Invoice Date,Stock Code,Quantity,Transaction Id,Product_Sku,Product,Product Catagory,Average Price,Revenue,Tax,Delivery Cost
0,536598,2017-01-01,21421,1,16736,GGOENEBJ079499,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,153.71,149.0,10.89,6.5
1,536598,2017-01-01,21421,1,16733,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,Nest-USA,122.77,119.0,11.00,6.5
2,536598,2017-01-01,21421,1,16732,GGOENEBJ079499,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,153.71,149.0,13.22,6.5
3,536598,2017-01-01,21421,1,16731,GGOENEBQ078999,Nest® Cam Outdoor Security Camera - USA,Nest-USA,122.77,238.0,20.80,6.5
4,536598,2017-01-01,21421,1,16727,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,Nest-USA,122.77,119.0,11.00,6.5


In [32]:
last_join = pd.merge(cleaned_retail_online,marketing_transf, on='Invoice Date',how = 'inner')
last_join.head(5)

,Invoice No,Invoice Date,Stock Code,Quantity,Transaction Id,Product_Sku,Product,Product Catagory,Average Price,Revenue,Tax,Delivery Cost,Offline Budget,Online Budget
0,536598,2017-01-01,21421,1,16736,GGOENEBJ079499,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,153.71,149.0,10.89,6.5,4500,2424.5
1,536598,2017-01-01,21421,1,16733,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,Nest-USA,122.77,119.0,11.00,6.5,4500,2424.5
2,536598,2017-01-01,21421,1,16732,GGOENEBJ079499,Nest® Learning Thermostat 3rd Gen-USA - Stainl...,Nest-USA,153.71,149.0,13.22,6.5,4500,2424.5
3,536598,2017-01-01,21421,1,16731,GGOENEBQ078999,Nest® Cam Outdoor Security Camera - USA,Nest-USA,122.77,238.0,20.80,6.5,4500,2424.5
4,536598,2017-01-01,21421,1,16727,GGOENEBB078899,Nest® Cam Indoor Security Camera - USA,Nest-USA,122.77,119.0,11.00,6.5,4500,2424.5


In [ ]:
# ignore below-------------------------------------

In [55]:
# engine.execute("""select * from pragma_table_info('online');""").fetchall()

[(0, '\ufeffTransaction ID', '', 0, None, 0),
 (1, 'Date', '', 0, None, 0),
 (2, 'Product SKU', '', 0, None, 0),
 (3, 'Product', '', 0, None, 0),
 (4, 'Product Category (Enhanced E-commerce)', '', 0, None, 0),
 (5, 'Quantity', '', 0, None, 0),
 (6, 'Avg. Price', '', 0, None, 0),
 (7, 'Revenue', '', 0, None, 0),
 (8, 'Tax', '', 0, None, 0),
 (9, 'Delivery', '', 0, None, 0)]

In [86]:
# engine.execute("""select * from pragma_table_info('key_sku');""").fetchall()

[(0, '\ufeffProduct SKU', '', 0, None, 0), (1, 'StockCode', '', 0, None, 0)]

In [57]:
# engine.execute("""select * from pragma_table_info('retail');""").fetchall()

[(0, 'InvoiceNo', '', 0, None, 0),
 (1, 'InvoiceDate', '', 0, None, 0),
 (2, 'StockCode', '', 0, None, 0),
 (3, 'Quantity', '', 0, None, 0)]

In [68]:
# engine.execute("""select * from pragma_table_info('marketing_spend');""").fetchall()

[(0, 'InvoiceDate', '', 0, None, 0),
 (1, 'Offline Budget', '', 0, None, 0),
 (2, 'Online Budget', '', 0, None, 0)]

In [70]:
# join1 =  engine.execute('SELECT * FROM marketing_spend inner join retail on marketing_spend.InvoiceDate = retail.InvoiceDate LIMIT 5').fetchall()

In [76]:
# for row in join1:
#     print(row)

('2017-01-01', '4500', '2424.5', '536598', '2017-01-01', '21421', '1')
('2017-01-01', '4500', '2424.5', '536598', '2017-01-01', '21422', '2')
('2017-01-01', '4500', '2424.5', '536598', '2017-01-01', '22178', '26')
('2017-01-01', '4500', '2424.5', '536599', '2017-01-01', '20749', '2')
('2017-01-01', '4500', '2424.5', '536599', '2017-01-01', '21056', '2')


In [111]:
# join1 =  engine.execute("""SELECT * FROM marketing_spend 
#                         inner join retail 
#                         on marketing_spend.InvoiceDate = retail.InvoiceDate 
#                         inner join key_SKU
#                         on key_sku.StockCode =retail.StockCode 
#                         inner join online
#                         on online.'ProductSKU' = KEY_SKU.'\ufeffProduct SKU' limit 1""").fetchall()

In [ ]:
#need to figure out how to make this look nicely?

In [112]:
# for row in join1:
#     print(row)

('2017-01-01', '4500', '2424.5', '536598', '2017-01-01', '21421', '1', 'GGOENEBQ079099', '21421', '16699', '20170101', 'GGOENEBQ079099', 'Nest® Protect Smoke + CO White Battery Alarm-USA', 'Nest-USA', '1', '81.5', '79', '7.28', '6.5')


In [63]:
# engine2.execute('SELECT * FROM key_sku inner join').fetchall()

In [64]:
# engine3.execute('SELECT * FROM retail LIMIT 5').fetchall()

In [65]:
# engine4.execute('select * from marketing_spend').fetchall()

In [66]:
# engine.execute('SELECT * FROM online LIMIT 5').fetchall()

[('48497', '20171231', 'GGOENEBQ079099', 'Nest® Protect Smoke + CO White Battery Alarm-USA', 'Nest-USA', '4', '80.52', '316', '34.44', '19.99'),
 ('48496', '20171231', 'GGOENEBQ079199', 'Nest® Protect Smoke + CO White Wired Alarm-USA', 'Nest-USA', '5', '80.52', '395', '33.14', '6.5'),
 ('48495', '20171231', 'GGOENEBQ084699', 'Nest® Learning Thermostat 3rd Gen-USA - White', 'Nest-USA', '1', '151.88000000000002', '149', '12.06', '6.5'),
 ('48494', '20171231', 'GGOEGAEB091117', 'Google Zip Hoodie Black', 'Apparel', '1', '48.92', '47.99', '3.82', '6.5'),
 ('48493', '20171231', 'GGOENEBB078899', 'Nest® Cam Indoor Security Camera - USA', 'Nest-USA', '1', '121.3', '119', '8.31', '6.5')]

InvalidRequestError: SQL expression, column, or mapped entity expected - got '[('2017-01-01', '4500', '2424.5', '536598', '2017-01-01', '21421', '1', 'GGOENEBQ079099', '21421', '16699', '20170101', 'GGOENEBQ079099', 'Nest® Protect Smoke + CO White Battery Alarm-USA', 'Nest-USA', '1', '81.5', '79', '7.28', '6.5'), ('2017-01-01', '4500', '2424.5', '536598', '2017-01-01', '21421', '1', 'GGOENEBQ079099', '21421', '16703', '20170101', 'GGOENEBQ079099', 'Nest® Protect Smoke + CO White Battery Alarm-USA', 'Nest-USA', '2', '81.5', '158', '0', '6.5'), ('2017-01-01', '4500', '2424.5', '536598', '2017-01-01', '21421', '1', 'GGOENEBQ079099', '21421', '16776', '20170102', 'GGOENEBQ079099', 'Nest® Protect Smoke + CO White Battery Alarm-USA', 'Nest-USA', '2', '81.5', '158', '15.229999999999999', '6.5'), ('2017-01-01', '4500', '2424.5', '536598', '2017-01-01', '21421', '1', 'GGOENEBQ079099', '21421', '16777', '20170102', 'GGOENEBQ079099', 'Nest® Protect Smoke + CO White Battery Alarm-USA', 'Nest-USA',

In [113]:
# print(join1)

[('2017-01-01', '4500', '2424.5', '536598', '2017-01-01', '21421', '1', 'GGOENEBQ079099', '21421', '16699', '20170101', 'GGOENEBQ079099', 'Nest® Protect Smoke + CO White Battery Alarm-USA', 'Nest-USA', '1', '81.5', '79', '7.28', '6.5')]
